In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import tempfile
from tqdm import tqdm
from feat import Detector
from PIL import Image
import mediapipe as mp

# Initialize detectors
detector = Detector(face_model="retinaface", landmark_model="mobilefacenet", au_model="svm")
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# AU columns
au_base = [
    "AU01", "AU02", "AU04", "AU05", "AU06", "AU07", "AU09", "AU10", "AU11", "AU12",
    "AU14", "AU15", "AU17", "AU20", "AU23", "AU24", "AU25", "AU26", "AU28", "AU43"
]
au_columns = [f"{i}_{au}" for i in range(1, 4) for au in au_base]

# MediaPipe landmark column names
def generate_mediapipe_columns(label, frame_idx, count):
    return [f"{frame_idx}_{label}_{i}_{coord}" for i in range(count) for coord in ["x", "y", "z"]]

body_columns = generate_mediapipe_columns("body", 1, 33) + generate_mediapipe_columns("body", 2, 33) + generate_mediapipe_columns("body", 3, 33)
lh_columns   = generate_mediapipe_columns("lh", 1, 21) + generate_mediapipe_columns("lh", 2, 21) + generate_mediapipe_columns("lh", 3, 21)
rh_columns   = generate_mediapipe_columns("rh", 1, 21) + generate_mediapipe_columns("rh", 2, 21) + generate_mediapipe_columns("rh", 3, 21)

all_columns = ["filename", "relative_path"] + au_columns + body_columns + lh_columns + rh_columns
all_data = []

# Function to save frame and extract AUs
def extract_aus_from_frame(frame):
    with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmpfile:
        tmp_path = tmpfile.name
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        Image.fromarray(rgb_frame).save(tmp_path)
    feat_result = detector.detect_image(tmp_path)
    os.remove(tmp_path)
    if feat_result.empty:
        return [None] * len(au_base)
    else:
        return [feat_result.iloc[0][au] for au in au_base]

# Function to extract MediaPipe landmarks
def extract_mediapipe_landmarks_from_frame(frame):
    results = holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    def extract_landmarks(landmarks, expected_count):
        if landmarks:
            return [coord for lm in landmarks.landmark for coord in (lm.x, lm.y, lm.z)]
        else:
            return [None] * (expected_count * 3)
    body = extract_landmarks(results.pose_landmarks, 33)
    lh = extract_landmarks(results.left_hand_landmarks, 21)
    rh = extract_landmarks(results.right_hand_landmarks, 21)
    return body, lh, rh

# Function to find all video files recursively
def find_video_files(root_directory):
    video_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.m4v')
    video_files = []
    
    for root, dirs, files in os.walk(root_directory):
        for file in files:
            if file.lower().endswith(video_extensions):
                full_path = os.path.join(root, file)
                relative_path = os.path.relpath(full_path, root_directory)
                video_files.append((full_path, relative_path, file))
    
    return video_files

# Process all videos in directory and subdirectories
root_video_folder = "Train"  # Change this to your root directory name
print(f"Searching for video files in '{root_video_folder}' and subdirectories...")

video_files = find_video_files(root_video_folder)
print(f"Found {len(video_files)} video files")

if len(video_files) == 0:
    print("No video files found. Please check the directory path and file extensions.")
    exit()

with mp_holistic.Holistic(static_image_mode=True) as holistic:
    for full_path, relative_path, filename in tqdm(video_files, desc="Processing videos"):
        try:
            cap = cv2.VideoCapture(full_path)
            
            # Check if video opened successfully
            if not cap.isOpened():
                print(f"Warning: Could not open video {relative_path}")
                # Add row with None values for failed videos
                row = [filename, relative_path] + [None] * (len(au_columns) + len(body_columns) + len(lh_columns) + len(rh_columns))
                all_data.append(row)
                continue
            
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
            # Handle videos with very few frames
            if frame_count < 3:
                target_indices = list(range(frame_count))
                # Pad with last frame if needed
                while len(target_indices) < 3:
                    target_indices.append(frame_count - 1 if frame_count > 0 else 0)
            else:
                target_indices = [0, frame_count // 2, frame_count - 1]

            frames = []
            for idx in target_indices:
                cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                ret, frame = cap.read()
                frames.append(frame if ret else None)
            cap.release()

            # Extract features
            row = [filename, relative_path]
            for i, frame in enumerate(frames, start=1):
                if frame is not None:
                    row.extend(extract_aus_from_frame(frame))
                    body, lh, rh = extract_mediapipe_landmarks_from_frame(frame)
                    row.extend(body)
                    row.extend(lh)
                    row.extend(rh)
                else:
                    row.extend([None] * (len(au_base) + 33*3 + 21*3 + 21*3))
            all_data.append(row)
            
        except Exception as e:
            print(f"Error processing {relative_path}: {str(e)}")
            # Add row with None values for failed videos
            row = [filename, relative_path] + [None] * (len(au_columns) + len(body_columns) + len(lh_columns) + len(rh_columns))
            all_data.append(row)

# Save to CSV
df = pd.DataFrame(all_data, columns=all_columns)
df.to_csv("features.csv", index=False)
print("✅ Saved features.csv with", len(df), "rows")

# Print some statistics
successful_rows = df[df[au_columns[0]].notna()].shape[0]
failed_rows = len(df) - successful_rows
print(f"Successfully processed: {successful_rows} videos")
print(f"Failed to process: {failed_rows} videos")

c:\Users\PC\anaconda3\envs\fb-tracking\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Searching for video files in 'Train' and subdirectories...
Found 297 video files


Processing videos: 100%|██████████| 297/297 [17:14<00:00,  3.48s/it]

✅ Saved features.csv with 297 rows
Successfully processed: 297 videos
Failed to process: 0 videos
